# APIM ❤️ AI Foundry

## AI Foundry Deepseek lab
![flow](../../images/ai-foundry-deepseek.gif)

Playground to try the [Deepseek R1 model](https://azure.microsoft.com/en-us/blog/deepseek-r1-is-now-available-on-azure-ai-foundry-and-github/) via the AI Model Inference from [Azure AI Foundry](https://azure.microsoft.com/en-us/products/ai-foundry). This lab uses the [Azure AI Model Inference API](https://learn.microsoft.com/en-us/azure/ai-foundry/model-inference/how-to/inference?tabs=python) and two APIM LLM policies: [llm-token-limit](https://learn.microsoft.com/en-us/azure/api-management/llm-token-limit-policy) and [llm-emit-token-metric](https://learn.microsoft.com/en-us/azure/api-management/llm-emit-token-metric-policy).

[View policy configuration](policy.xml)

### Result

![result](result.png)

### Prerequisites

- [Python 3.12 or later version](https://www.python.org/) installed
- [VS Code](https://code.visualstudio.com/) installed with the [Jupyter notebook extension](https://marketplace.visualstudio.com/items?itemName=ms-toolsai.jupyter) enabled
- [Python environment](https://code.visualstudio.com/docs/python/environments#_creating-environments) with the [requirements.txt](../../requirements.txt) or run `pip install -r requirements.txt` in your terminal
- [An Azure Subscription](https://azure.microsoft.com/free/) with [Contributor](https://learn.microsoft.com/en-us/azure/role-based-access-control/built-in-roles/privileged#contributor) + [RBAC Administrator](https://learn.microsoft.com/en-us/azure/role-based-access-control/built-in-roles/privileged#role-based-access-control-administrator) or [Owner](https://learn.microsoft.com/en-us/azure/role-based-access-control/built-in-roles/privileged#owner) roles
- [Azure CLI](https://learn.microsoft.com/cli/azure/install-azure-cli) installed and [Signed into your Azure subscription](https://learn.microsoft.com/cli/azure/authenticate-azure-cli-interactively)

▶️ Click `Run All` to execute all steps sequentially, or execute them `Step by Step`... 


<a id='0'></a>
### 0️⃣ Initialize notebook variables

- Resources will be suffixed by a unique string based on your subscription id.
- Adjust the location parameters according your preferences and on the [product availability by Azure region.](https://azure.microsoft.com/explore/global-infrastructure/products-by-region/?cdn=disable&products=cognitive-services,api-management) 
- Adjust the OpenAI model and version according the [availability by region.](https://learn.microsoft.com/azure/ai-services/openai/concepts/models) 

In [ ]:
import os, sys, json
sys.path.insert(1, '../../shared')  # add the shared directory to the Python path
import utils

deployment_name = os.path.basename(os.path.dirname(globals()['__vsc_ipynb_file__']))
resource_group_name = f"lab-{deployment_name}" # change the name to match your naming style
resource_group_location = "eastus2"

aiservices_config = [{"name": "foundry1", "location": "eastus2"}]

models_config = [{"name": "DeepSeek-R1", "publisher": "DeepSeek", "version": "1", "sku": "GlobalStandard", "capacity": 1},
                 {"name": "Phi-4", "publisher": "Microsoft", "version": "3", "sku": "GlobalStandard", "capacity": 1}]

apim_sku = 'Basicv2'
apim_subscriptions_config = [{"name": "subscription1", "displayName": "Subscription 1"}, 
                             {"name": "subscription2", "displayName": "Subscription 2"}, 
                             {"name": "subscription3", "displayName": "Subscription 3"}]

inference_api_path = "inference"  # path to the inference API in the APIM service
inference_api_type = "AzureAI"  # options: AzureOpenAI, AzureAI, OpenAI, PassThrough
inference_api_version = "2024-05-01-preview"
foundry_project_name = deployment_name

utils.print_ok('Notebook initialized')

<a id='1'></a>
### 1️⃣ Verify the Azure CLI and the connected Azure subscription

The following commands ensure that you have the latest version of the Azure CLI and that the Azure CLI is connected to your Azure subscription.

In [ ]:
output = utils.run("az account show", "Retrieved az account", "Failed to get the current az account")

if output.success and output.json_data:
    current_user = output.json_data['user']['name']
    tenant_id = output.json_data['tenantId']
    subscription_id = output.json_data['id']

    utils.print_info(f"Current user: {current_user}")
    utils.print_info(f"Tenant ID: {tenant_id}")
    utils.print_info(f"Subscription ID: {subscription_id}")

<a id='2'></a>
### 2️⃣ Create deployment using 🦾 Bicep

This lab uses [Bicep](https://learn.microsoft.com/azure/azure-resource-manager/bicep/overview?tabs=bicep) to declarative define all the resources that will be deployed in the specified resource group. Change the parameters or the [main.bicep](main.bicep) directly to try different configurations. 

In [ ]:
# Create the resource group if doesn't exist
utils.create_resource_group(resource_group_name, resource_group_location)

# Define the Bicep parameters
bicep_parameters = {
    "$schema": "https://schema.management.azure.com/schemas/2019-04-01/deploymentParameters.json#",
    "contentVersion": "1.0.0.0",
    "parameters": {
        "apimSku": { "value": apim_sku },
        "aiServicesConfig": { "value": aiservices_config },
        "modelsConfig": { "value": models_config },
        "apimSubscriptionsConfig": { "value": apim_subscriptions_config },
        "inferenceAPIPath": { "value": inference_api_path },
        "inferenceAPIType": { "value": inference_api_type },
        "foundryProjectName": { "value": foundry_project_name }
    }
}

# Write the parameters to the params.json file
with open('params.json', 'w') as bicep_parameters_file:
    bicep_parameters_file.write(json.dumps(bicep_parameters))

# Run the deployment
output = utils.run(f"az deployment group create --name {deployment_name} --resource-group {resource_group_name} --template-file main.bicep --parameters params.json",
    f"Deployment '{deployment_name}' succeeded", f"Deployment '{deployment_name}' failed")

<a id='3'></a>
### 3️⃣ Get the deployment outputs

Retrieve the required outputs from the Bicep deployment.

In [ ]:
# Obtain all of the outputs from the deployment
output = utils.run(f"az deployment group show --name {deployment_name} -g {resource_group_name}", f"Retrieved deployment: {deployment_name}", f"Failed to retrieve deployment: {deployment_name}")

if output.success and output.json_data:
    log_analytics_id = utils.get_deployment_output(output, 'logAnalyticsWorkspaceId', 'Log Analytics Id')
    apim_service_id = utils.get_deployment_output(output, 'apimServiceId', 'APIM Service Id')
    apim_resource_gateway_url = utils.get_deployment_output(output, 'apimResourceGatewayURL', 'APIM API Gateway URL')
    apim_subscriptions = json.loads(utils.get_deployment_output(output, 'apimSubscriptions').replace("\'", "\""))
    for subscription in apim_subscriptions:
        subscription_name = subscription['name']
        subscription_key = subscription['key']
        utils.print_info(f"Subscription Name: {subscription_name}")
        utils.print_info(f"Subscription Key: ****{subscription_key[-4:]}")
    api_key = apim_subscriptions[0].get("key") # default api key to the first subscription key

<a id='sdk'></a>
### 🧪 Test the API using the Inference SDK


In [ ]:
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

client = ChatCompletionsClient(
    endpoint=f"{apim_resource_gateway_url}/{inference_api_path}/models",
    credential=AzureKeyCredential(apim_subscriptions[0]['key']),
)

response = client.complete(
    messages=[
        SystemMessage(content="You are a sarcastic, unhelpful assistant."),
        UserMessage(content="Can you tell me the time, please?")
    ],
    max_tokens=2048,
    model=models_config[0]['name']
)

print(response.choices[0].message.content)

<a id='ratelimit'></a>
### 🧪 Test the token rate limit


In [ ]:
import requests

url = f"{apim_resource_gateway_url}/{inference_api_path}/models/chat/completions?api-version={inference_api_version}"
api_runs = []
for i in range(10):
    messages={"messages":[
            {"role": "system", "content": "You are a sarcastic, unhelpful assistant."},
            {"role": "user", "content": "Can you tell me the time, please?"}
        ],
        "model": models_config[0]['name']
    }
    response = requests.post(url, headers={'api-key': apim_subscriptions[0]['key']}, json=messages)
    if (response.status_code == 200):
        data = json.loads(response.text)
        total_tokens = data.get("usage").get("total_tokens")
        print("▶️ Run: ", i+1, "status code: ", response.status_code, "✅", " tokens: ", total_tokens)
        print("💬 ", data.get("choices")[0].get("message").get("content"))
    else:
        print("▶️ Run: ", i+1, "status code: ", response.status_code, "⛔")
        print(response.text)
        total_tokens = 0
    api_runs.append((total_tokens, response.status_code))


<a id='plot'></a>
### 🔍 Analyze Token Rate limiting results

In [ ]:
# plot the results
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = [15, 7]
df = pd.DataFrame(api_runs, columns=['Tokens', 'Status Code'])
df['Run'] = range(1, len(df) + 1)
colors = ['red' if str(code).startswith('5') else 'yellow' if str(code).startswith('4') else 'lightblue' for code in df['Status Code']]
ax = df.plot(kind='bar', x='Run', y='Tokens', color=colors, legend=False)
plt.title('Rate Limiting results')
plt.xlabel('Runs')
plt.ylabel('Tokens')
plt.xticks(df['Run'], rotation=0)
for i, val in enumerate(df['Status Code']):
    ax.text(i, 20, '' if int(val) == 200 else 'Status:429', ha='center', va='bottom')
for i, val in enumerate(df['Tokens']):
    ax.text(i, df['Tokens'][i] + 5, '' if int(val) == 0 else val, ha='center', va='bottom')
accumulated_tokens = df['Tokens'].cumsum()
ax.plot(df['Run']-1, accumulated_tokens, color='green', label='Accumulated Tokens')
for i, val in enumerate(accumulated_tokens):
    ax.text(i, val + 6, str(int(val)), ha='center', va='bottom', label='Accumulated Tokens')
plt.show()

<a id='ratelimit'></a>
### 🧪 Test the emit token metric with the configured subscriptions


In [ ]:
import random
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

for i in range(10):
    client = ChatCompletionsClient(
        endpoint=f"{apim_resource_gateway_url}/{inference_api_path}/models",
        credential=AzureKeyCredential(random.choice(apim_subscriptions)['key']),
    )
    response = client.complete(
        messages=[
            SystemMessage(content="You are a sarcastic, unhelpful assistant."),
            UserMessage(content="Can you tell me the time, please?")
        ],
        max_tokens=2048,
        model=models_config[0]['name']
    )
    print("▶️ Run: ", i+1, "🗨️\n", response.choices[0].message.content)

<a id='kql'></a>
### 🔍 Analyze LL Logging with a KQL query

Note that it may take a few minutes for data to become available.

In [ ]:
import pandas as pd

query = "let llmHeaderLogs = ApiManagementGatewayLlmLog \
| where DeploymentName != ''; \
let llmLogsWithSubscriptionId = llmHeaderLogs \
| join kind=leftouter ApiManagementGatewayLogs on CorrelationId \
| project \
    TimeGenerated, SubscriptionId = ApimSubscriptionId, DeploymentName, TotalTokens; \
llmLogsWithSubscriptionId \
| summarize \
    SumTotalTokens      = sum(TotalTokens) \
  by bin(TimeGenerated, 1m), SubscriptionId, DeploymentName \
  | order by TimeGenerated asc"

output = utils.run(f"az monitor log-analytics query -w {log_analytics_id} --analytics-query \"{query}\"", "Retrieved log analytics query output", "Failed to retrieve log analytics query output") 
if output.success and output.json_data:
    table = output.json_data
    display(pd.DataFrame(table))



<a id='plot'></a>
### 🔍 Plot the custom metrics results

In [ ]:
import pandas as pd

# Convert table to DataFrame
df_table = pd.DataFrame(table)

# Convert SumTotalTokens to numeric
df_table['SumTotalTokens'] = pd.to_numeric(df_table['SumTotalTokens'], errors='coerce')

# Convert TimeGenerated to readable time (HH:MM)
df_table['TimeGenerated'] = pd.to_datetime(df_table['TimeGenerated']).dt.strftime('%H:%M')

# Pivot the DataFrame
df_pivot = df_table.pivot(index='TimeGenerated', columns='SubscriptionId', values='SumTotalTokens')

# Plot the pivoted DataFrame
ax = df_pivot.plot(kind='bar', figsize=(15, 7))
ax.set_title('SumTotalTokens by SubscriptionId and TimeGenerated')
ax.set_xlabel('TimeGenerated')
ax.set_ylabel('SumTotalTokens')
ax.legend(title='SubscriptionId')

<a id='portal'></a>
### 🔍 See the metrics on the Azure Portal

Open the Application Insights resource, navigate to the Metrics blade, then select the defined namespace (llm). Choose the metric "Total Tokens" with a Sum aggregation. Then, apply splitting by 'Subscription Id' to view values for each dimension.


<a id='clean'></a>
### 🗑️ Clean up resources

When you're finished with the lab, you should remove all your deployed resources from Azure to avoid extra charges and keep your Azure subscription uncluttered.
Use the [clean-up-resources notebook](clean-up-resources.ipynb) for that.